In [1]:
import icalendar

In [2]:
import pytz, zoneinfo, dateutil.tz  # timezone libraries
import datetime, icalendar
import pyaml
import pandas as pd 

In [3]:
import yaml
from datetime import datetime, timedelta
import re
from ics import Calendar, Event



def convert_datestring (datestring):
    format = "%m/%d/%Y"
    date_object = datetime.strptime(datestring, format)
    return date_object


def get_start_end_dates(block_id, my_dict):
    start, end = (my_dict['block_dates'][block_id].split())
    start = convert_datestring(start)
    end = convert_datestring(end)
    return start, end


def date_lists(start_date, end_date):
    current_date = start_date
    dates = []
    while current_date <= end_date:
        dates.append(current_date)
        current_date += timedelta(days=1)
    return dates
    
    
def get_specific_rotation_schedule(block_id, rotation, my_dict):
    rotation = re.sub('\s', '', rotation)
    

        
    try:
        rotation, individual = rotation.split('-')
    except:
        rotation = rotation
        individual = 'A'
        
    if re.search('Janeway|Barker|Longcope|Thayer', rotation):
        rotation_category = 'O'    
    elif re.search('CJ|Polk|MEG', rotation):
        rotation_category = 'Subspecialty'    
        
    elif re.search('PCCU|Addiction', rotation):
        # these rotation have MTWTF rotation with weekends off
        rotation_category = 'Ambulatory'
    else:
        rotation_category = rotation
        
    for schedule_type, schedule in my_dict[rotation_category].items():
        if block_id in schedule['blocks']:
            return schedule[individual].split()
        
        
        
def block_dates (block_id, my_dict):
    """get start and end date for each block by going through the master yaml """
    start_date, end_date = (get_start_end_dates(block_id, my_dict))
    block_duration = date_lists(start_date, end_date)    
    return block_duration
 

def add_date_postcall_into_next_block(call_cycle, block_days):
    print(call_cycle[-1])
    if re.search(r'^\*', call_cycle[-1]):
        # Step 1: Get the last date in the list
        last_date = block_days[-1]

        # Step 2: Calculate the next date (assuming consecutive days, so add 1 day)
        next_date = last_date + timedelta(days=1)

        # Step 3: Add the new date to the list
        block_days.append(next_date)
    return block_days
    
    
def create_daily_inpatient_schedule(block_id, rotation, my_dict):
    """creates a inpatient schedule"""
    

In [4]:
# Read master tables

    
with open("/Users/cjyoon/Dropbox/Osler/Osler_shiny/call_schedule_master.yaml") as f:
    my_dict = yaml.safe_load(f)

    
master_schedule = pd.read_excel('~/Dropbox/Osler/Intern Schedule 2024-2025.xlsx', sheet_name='Schedule')
resident_names = master_schedule.iloc[:, 0].to_list()
series = pd.Series(resident_names)

# Remove NaN values
cleaned_series = series.dropna()

# Convert back to a list
resident_names = cleaned_series.tolist()
print(resident_names)

['Alexander_Mathew', 'Ancha_Bhavya', 'Arahirwa_Victor', 'Bothwell_James', 'Broshkevitch_Anna', 'Bullock_Avery', 'Carter_Julia', 'Kesaf_Ayse', 'Chabaan_Lara', 'Chang_Michael', 'Clinton_Ivor', 'Ermolovich_Jake', 'Fatteh_Maria', 'Goren_Lea', 'Guo_Matthew', 'Holler_Albert', 'Jabboure_Fayez', 'Cazimir_Catherine', 'Khunsriraksakul_Chachrit', 'Kim_Minjoo', 'Kodukula_Sai', 'Koul_Kayshap', 'Kuzma_Irena', 'Littman_Emily', 'Lopez_Christina', 'Lutz_Lydia', 'Mathias_Trevor', 'Mhaimeed_Nour', 'Mollenkopf_Sarah', 'Moore_Erica', 'Morcos_George', 'Murphy_Cara', 'Ozkan_Bige', 'Payne_Lauren', 'Pazzi_Carlotta', 'Potharazu_Archit', 'Rao_Rohit', 'Reddy_Tina', 'Riddick_Erica', 'Rodriguez-Fernandez_Monica', 'Lopez-Silva_Carolina', 'Suresh_Abhilash', 'Tarasco_Alexander', 'Tracey_Evan', 'Vasoya_Roshan', 'Weinstein_Robert', 'Ye_Congxie', 'Yeudall_Scott', 'Yoon_Christopher', 'Zarker_Andrew', 'Ziogos_Timos', 'ED_MICU', 'ED_CCU', 'OPH_Morello_Tim', 'OPH_Kim_Suzie', 'OPH_Shah_Rohan', 'OPH_Ali_Muhammad', 'OPH_Pan-Doh

In [5]:
def get_schedule(resident_name, master_schedule):
    """gets the specific intern's schedule from the table"""
    filtered_df = master_schedule[master_schedule.iloc[:, 0] == resident_name]
    return (filtered_df.squeeze().to_list()[1:])



In [6]:
new_column_names = [
    f"{col} {master_schedule.iloc[0, idx].strftime('%m/%d')}-{master_schedule.iloc[1, idx].strftime('%m/%d')}"
    if not isinstance(master_schedule.iloc[0, idx], float) and not isinstance(master_schedule.iloc[1, idx], float) 
    else f"{col}"
    for idx, col in enumerate(master_schedule.columns) if not isinstance(col, float)
]

# Update the column names in the DataFrame
master_schedule.columns = new_column_names
print(new_column_names)


master_table_tsv = master_schedule.iloc[3:].fillna('')
master_table_tsv.columns.values[0] = 'name'
master_table_tsv.to_csv('/Users/cjyoon/Dropbox/Osler/Osler_shiny/schedule/block_view.tsv', sep='\t', index=False)

['Unnamed: 0', '1A 07/01-07/10', '1B 07/11-07/24', '2A 07/25-08/07', '2B 08/08-08/21', '3A 08/22-09/04', '3B 09/05-09/18', '4A 09/19-10/02', '4B 10/03-10/16', '5A 10/17-10/30', '5B 10/31-11/13', '6A 11/14-11/27', '6B 11/28-12/11', '7A 12/12-12/19', 'HOLIDAY 1 12/20-12/26', 'HOLIDAY 2 12/27-01/02', '7B 01/03-01/15', '8A 01/16-01/29', '8B 01/30-02/12', '9A 02/13-02/26', '9B 02/27-03/12', '10A 03/13-03/26', '10B 03/27-04/09', '11A 04/10-04/23', '11B 04/24-05/07', '12A 05/08-05/21', '12B 05/22-06/04', '13A 06/05-06/18', '13B 06/19-06/30']


In [7]:
july1_offset = my_dict['july1_offset']
# Create a calendar and event


for resident_name in resident_names:
    date_list = []
    schedule_list = []        
    my_blocks = (get_schedule(resident_name, master_schedule))
    print(my_blocks)
    

    calendar = Calendar()
    event = Event()
    with open(f"/Users/cjyoon/Dropbox/Osler/Osler2024_schedule/{resident_name}_schedule_2024_only.tsv", "w") as g:
        g.write('name\tdate\tschedule\n')
        for block_id, rotation in zip(my_dict['block_dates'].keys(), my_blocks):
            # only write out upto holiday block 2 since remaining scheudle has not been finalized yet
            if block_id in ['1A', '1B', '2A', '2B', '3A', '3B', '4A', '4B', '5A', '5B', '6A', '6B', '7A', 'Holiday 1', "Holiday 2"]:

                rotation = re.sub('\s', '', rotation)

                print(block_id, rotation)

                block_days = block_dates(block_id, my_dict)        

                try:
                    call_cycle = get_specific_rotation_schedule(block_id, rotation, my_dict)
                except:
                    # if specific call cycle is unknown for a rotation, then default to unknown so manual creation can be made
                    call_cycle = ['UNKNOWN'] * 14


                print(f'{block_days} {len(block_days)}')
                print(f'{call_cycle} {len(call_cycle)}')


                if block_id == '1A':
                    # create call cycle that is offset to account for different start date for july1 cycle
                    del call_cycle[:july1_offset]

                # check if block call cycle ends in a post call
                block_days = add_date_postcall_into_next_block(call_cycle, block_days)

                # Loop through each date and corresponding schedule
                for date, schedule in zip(block_days, call_cycle):
                    event = Event()  # Create a new event object inside the loop
                    event.name = f'{block_id} {rotation} {schedule}'
                    event.begin = date
                    event.make_all_day()  # Make the event an all-day event
                    print(date, block_id, rotation, schedule)
                    # Add event to calendar
                    calendar.events.add(event)
                    date_string = date.strftime('%m-%d-%Y')
                    g.write(f'{resident_name}\t{date_string}\t{event.name}\n')



        # Write the calendar to a .ics file
    with open(f"/Users/cjyoon/Dropbox/Osler/Osler2024_schedule/{resident_name}_schedule_2024_only.ics", "w") as f:
        f.writelines(calendar.serialize_iter())

    
    

['Ambulatory', 'MICU - F', 'Thayer - C', 'Thayer - C', 'Ambulatory', 'Brancati - B', 'Vacation', 'Subspecialty', 'Ambulatory', 'Thayer - C', 'Thayer - D', 'MPC - C', 'Ambulatory', 'Off', 'Thayer - B', 'Thayer', 'MTL', 'MICU', 'Ambulatory', 'MICU', 'Thayer', 'Thayer', 'Ambulatory', 'Vacation', 'Thayer', 'Thayer', 'Ambulatory', 'CCU']
1A Ambulatory
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['DAY', 'DAY', 'OFF', 'OFF', 'DAY', 'DAY', 'DAY', 'DAY', 'DAY', 'OFF', 'OFF', 'DAY', 'DAY', 'DAY'] 14
DAY
2024-07-01 00:00:00 1A Ambulatory DAY
2024-07-02 00:00:00 1A Ambulatory DAY
2024-07-03 00:00:00 1A Ambulatory DAY
2024-07-04 00:00:00 1A Ambulatory DAY
2024-07-05 00:00:0

['Ambulatory', 'MPC - A', 'Janeway - C', 'Janeway - C', 'Ambulatory', 'CCU - C', 'Subspecialty', 'MICU - A', 'Ambulatory', 'Vacation', 'Janeway - A', 'Janeway - A', 'Ambulatory', 'CCU - A', 'Off', 'Vacation', 'Janeway', 'Janeway', 'Ambulatory', 'Janeway', 'Janeway', 'MICU', 'Ambulatory', 'MTL', 'MPC', 'Brancati', 'Ambulatory', 'Janeway']
1A Ambulatory
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['DAY', 'DAY', 'OFF', 'OFF', 'DAY', 'DAY', 'DAY', 'DAY', 'DAY', 'OFF', 'OFF', 'DAY', 'DAY', 'DAY'] 14
DAY
2024-07-01 00:00:00 1A Ambulatory DAY
2024-07-02 00:00:00 1A Ambulatory DAY
2024-07-03 00:00:00 1A Ambulatory DAY
2024-07-04 00:00:00 1A Ambulatory DAY
2024-07-05 00

['Brancati - A', 'Ambulatory', 'Thayer - B', 'Thayer - B', 'MPC - B', 'Ambulatory', 'Thayer - C', 'Thayer - C', 'Vacation', 'Ambulatory', 'MICU - E', 'Thayer - D', 'Thayer - D', 'Off', 'MICU - B', 'Ambulatory', 'Leuks', 'Brancati', 'CCU', 'Ambulatory', 'Thayer', 'Thayer', 'MPC', 'Ambulatory', 'Vacation', 'MICU', 'Thayer', 'Ambulatory']
1A Brancati-A
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['DAY', 'DAY', 'DAY', 'OFF', 'NIGHT', 'NIGHT', 'NIGHT', 'POST', 'OFF', 'DAY', 'DAY', 'DAY', 'DAY', 'DAY'] 14
DAY
2024-07-01 00:00:00 1A Brancati-A NIGHT
2024-07-02 00:00:00 1A Brancati-A NIGHT
2024-07-03 00:00:00 1A Brancati-A NIGHT
2024-07-04 00:00:00 1A Brancati-A POST
2

2024-08-18 00:00:00 2B Janeway-A CALL
2024-08-19 00:00:00 2B Janeway-A POST
2024-08-20 00:00:00 2B Janeway-A DAY
2024-08-21 00:00:00 2B Janeway-A GOOD
3A MICU-A
[datetime.datetime(2024, 8, 22, 0, 0), datetime.datetime(2024, 8, 23, 0, 0), datetime.datetime(2024, 8, 24, 0, 0), datetime.datetime(2024, 8, 25, 0, 0), datetime.datetime(2024, 8, 26, 0, 0), datetime.datetime(2024, 8, 27, 0, 0), datetime.datetime(2024, 8, 28, 0, 0), datetime.datetime(2024, 8, 29, 0, 0), datetime.datetime(2024, 8, 30, 0, 0), datetime.datetime(2024, 8, 31, 0, 0), datetime.datetime(2024, 9, 1, 0, 0), datetime.datetime(2024, 9, 2, 0, 0), datetime.datetime(2024, 9, 3, 0, 0), datetime.datetime(2024, 9, 4, 0, 0)] 14
['GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', '*POST'] 15
*POST
2024-08-22 00:00:00 3A MICU-A GOOD
2024-08-23 00:00:00 3A MICU-A CALL
2024-08-24 00:00:00 3A MICU-A POST
2024-08-25 00:00:00 3A MICU-A OFF
2024-08-26 00:00:00 3A MICU-A CALL
2024

['MICU - F', 'MP Clinic', 'Longcope - C', 'Longcope - C', 'Peds', 'Peds', 'Peds', 'Peds', 'Longcope - A', 'Longcope - A', 'Brancati - D', 'MP Clinic', 'CCU - A', 'Off', 'MICU - D', 'Peds', 'Peds', 'Peds', 'Peds', 'Peds', 'Longcope', 'Vacation', 'MP Clinic', 'Longcope', 'Peds', 'Peds', 'Peds', 'Peds']
1A MICU-F
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', 'POST'] 14
POST
2024-07-01 00:00:00 1A MICU-F POST
2024-07-02 00:00:00 1A MICU-F GOOD
2024-07-03 00:00:00 1A MICU-F CALL
2024-07-04 00:00:00 1A MICU-F POST
2024-07-05 00:00:00 1A MICU-F OFF
2024-07-06 00:00:00 

['MICU - C', 'Leuks', 'Pulm (CJ)', 'Ambulatory', 'Barker - A', 'Barker - A', 'MPC - C', 'Ambulatory', 'Barker - B', 'Barker - B', 'Vacation', 'Ambulatory', 'MICU - C', 'MICU - B', 'Off', 'Brancati', 'CCU', 'Ambulatory', 'Barker', 'Barker', 'Vacation', 'Ambulatory', 'MPC', 'Solids', 'MICU', 'Ambulatory', 'Barker', 'Barker']
1A MICU-C
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['CALL', 'POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST', 'GOOD', 'CALL', 'POST', 'OFF', 'CALL', 'POST'] 14
POST
2024-07-01 00:00:00 1A MICU-C POST
2024-07-02 00:00:00 1A MICU-C OFF
2024-07-03 00:00:00 1A MICU-C CALL
2024-07-04 00:00:00 1A MICU-C POST
2024-07-05 00:00:00 1A MICU-C GO

2024-12-26 00:00:00 Holiday 1 MICU-C DAY
Holiday 2 Off
[datetime.datetime(2024, 12, 27, 0, 0), datetime.datetime(2024, 12, 28, 0, 0), datetime.datetime(2024, 12, 29, 0, 0), datetime.datetime(2024, 12, 30, 0, 0), datetime.datetime(2024, 12, 31, 0, 0), datetime.datetime(2025, 1, 1, 0, 0), datetime.datetime(2025, 1, 2, 0, 0)] 7
['OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF', 'OFF'] 14
OFF
2024-12-27 00:00:00 Holiday 2 Off OFF
2024-12-28 00:00:00 Holiday 2 Off OFF
2024-12-29 00:00:00 Holiday 2 Off OFF
2024-12-30 00:00:00 Holiday 2 Off OFF
2024-12-31 00:00:00 Holiday 2 Off OFF
2025-01-01 00:00:00 Holiday 2 Off OFF
2025-01-02 00:00:00 Holiday 2 Off OFF
['Ambulatory', 'MICU - C', 'MTL', 'Vacation', 'Ambulatory', 'MPC - A', 'Barker - B', 'Barker - B', 'Ambulatory', 'Barker - C', 'Barker - C', 'MICU - A', 'Ambulatory', 'Off', 'Barker - C', 'Barker', 'MICU', 'Subspecialty', 'Ambulatory', 'Solids', 'Vacation', 'Barker', 'Ambulatory', 'CCU', 'Barker', 'B

['Barker - C', 'Barker - C', 'Ambulatory', 'MICU - A', 'Brancati - B', 'Vacation', 'Ambulatory', 'MPC - B', 'Barker - D', 'Barker - D', 'Ambulatory', 'MICU - F', 'Barker - B', 'Barker - B', 'Off', 'Leuks', 'Ambulatory', 'MPC', 'Barker', 'Barker', 'Ambulatory', 'CCU', 'Brancati', 'Barker', 'Ambulatory', 'MICU', 'Vacation', 'Subspecialty']
1A Barker-C
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD'] 14
GOOD
2024-07-01 00:00:00 1A Barker-C DAY
2024-07-02 00:00:00 1A Barker-C GOOD
2024-07-03 00:00:00 1A Barker-C CALL
2024-07-04 00:00:00 1A Barker-C POST
2024-07-05 

['Janeway - D', 'Janeway - D', 'Ambulatory', 'Solids', 'MICU - F', 'Janeway - A', 'Ambulatory', 'MPC - A', 'Janeway - B', 'Subspecialty', 'Ambulatory', 'CCU - C', 'Vacation', 'Off', 'Janeway - A', 'Vacation', 'Ambulatory', 'Janeway', 'Janeway', 'MICU', 'Ambulatory', 'MICU', 'Janeway', 'Janeway', 'Ambulatory', 'Brancati', 'Leuks', 'MPC']
1A Janeway-D
[datetime.datetime(2024, 7, 1, 0, 0), datetime.datetime(2024, 7, 2, 0, 0), datetime.datetime(2024, 7, 3, 0, 0), datetime.datetime(2024, 7, 4, 0, 0), datetime.datetime(2024, 7, 5, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 7, 0, 0), datetime.datetime(2024, 7, 8, 0, 0), datetime.datetime(2024, 7, 9, 0, 0), datetime.datetime(2024, 7, 10, 0, 0)] 10
['POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY', 'GOOD', 'CALL', 'POST', 'DAY', 'OFF', 'CALL', 'POST', 'DAY'] 14
DAY
2024-07-01 00:00:00 1A Janeway-D POST
2024-07-02 00:00:00 1A Janeway-D DAY
2024-07-03 00:00:00 1A Janeway-D GOOD
2024-07-04 00:00:00 1A Janeway-D CALL
2024-07-

TypeError: expected string or bytes-like object, got 'float'

In [ ]:
def authenticate_google_drive():
    creds = None

    # Use token.json to store the user's access and refresh tokens
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If there are no valid credentials, let the user log in and authorize the app
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use the client_secret.json to start the OAuth flow
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/cjyoon/Dropbox/Osler/osler/client_secret_257342927366-4q3itidpqit3vcehjca5rvlg9m2evhsi.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for future runs in token.json
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Build the Google Drive service object
    service = build('drive', 'v3', credentials=creds)
    return service



def upload_ics_to_drive(file_path):
    service = authenticate_google_drive()

    # Upload the file to Google Drive with the correct MIME type for .ics files
    file_metadata = {'name': os.path.basename(file_path), 'mimeType': 'text/calendar'}
    media = MediaFileUpload(file_path, mimetype='text/calendar', parent='osler_schedule' )
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    # Make the file public and get the shareable link
    file_id = file.get('id')
    service.permissions().create(fileId=file_id, body={'type': 'anyone', 'role': 'reader'}).execute()
    shareable_url = f"https://drive.google.com/file/d/osler_schedule/{file_id}/view"
    
    return shareable_url


# for resident in resident_names:
#     file_path = f'/Users/cjyoon/Dropbox/Osler/osler/data/{resident}_schedule.ics'
#     shareable_link = upload_ics_to_drive(file_path)
#     print(f"{resident} schedule: {shareable_link}")